# Hagrid

## Materials

The goal for `hagrid` is to be a friendly helper in navigating the world of deployment and provisioning the `syft` stack and to take a multitude of complex steps and turn them into a handful of uniform and consistent commands.

`HAGrid` can deploy to a large variety of targets and by leveraging other tools and formats this can be extended. More specifically, `hagrid` can deploy to:
- `docker`
- any ip (`x.x.x.x`)
- `localhost`
- `azure`
- `gcp`
- `aws` (todo)
- `kubernetes` (coming soon)

### Launch to `Docker`

`hagrid` takes a small number of options and inputs from the user and translates them into lots of terminal commands and network calls as needed to setup and provision a node. In this case, they are `docker` and `docker compose` commands

```docker
docker compose
-p test_domain
--profile vpn
--profile blob-storage
--profile frontend
--env-file /Users/madhavajay/dev/PySyft/packages/grid/.envfile
--file docker-compose.yml
--file docker-compose.build.yml
--file docker-compose.dev.yml
up -d --build
```

In [ ]:
!hagrid launch test_domain domain to docker:8081 --tag=0.7.0 --dev --cmd

### Launch to any IP (`x.x.x.x`)

If we want to deploy `hagrid` to an existing `linux` server somewhere running `ssh`, we can ask `hagrid` to connect to it over `ssh` and do all the setup for us.

The way this works is by leveraging a tool called `ansible` which allows us to define the state in which we want a system to be in, and have it run commands called a `playbook` against that machine until it gets into the desired state.

```bash
!HAGRID_ART=false hagrid launch test_domain domain to 100.0.0.1 \
  --username=ubuntu --auth-type=key \
  --key-path=/Users/madhavajay/.ssh/azureuser.pem \
  --repo=OpenMined/PySyft --branch=0.7.0 --cmd
```

### Launch to `localhost`

All this does is use the same code path as the above `launch to x.x.x.x` except it passes in `--connection=local` which has the effect of telling `ansible` you won't need a username, password or ssh-key, just run locally and use `sudo` as necessary.

```bash
hagrid launch ${7} ${6} to localhost \
    --repo=${2} --branch=${3} \
    --release=${RELEASE} --tag=${DOCKER_TAG}
```

### Launch to Azure

Now that we know how `hagrid` can setup a linux machine for us, let's look at how we might use this to deploy a `cloud vm` onto `azure`.

First, you need to have the `azure-cli` tools which have many python packages 

```bash
hagrid launch test_domain domain to azure
```

### Launch to GCP

```bash
hagrid launch test_domain domain to gcp 
```

## Trial of the Flesh

#### 1. Deploy the `network` node either on `docker` or another `vm` somewhere on on-prem infrastructure or the `cloud`

Launching the `network` to `docker`


In [ ]:
!hagrid launch test_network network to docker:8082 --tag=0.7.0 --tail --dev

Check the `network` health

In [20]:
import syft as sy
import hagrid

hagrid.check('localhost:8082')

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:8082/login               │ ✅ │
│ api       │ http://localhost:8082/api/v1/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

Log in to the `network` and check its initial `domains`

In [2]:
network = sy.login('localhost', 8082)
network.domains

Connecting to localhost... done! 	 Logging into test_network... as GUEST...done!


""


In [9]:
domain_client = sy.login(email='my@email.com', password='password', url='localhost', port=8081, timeout=30)

Connecting to localhost...
ReadTimeout:
	Connection to node with url: localhost:8081 timed out after 30 seconds.
	Please try the following options:
	- Please try increasing the timeout by passing it as an argument to the login method.
	e.g. `sy.login(email='my@email.com', password='password', url='localhost', timeout=30)`
	- The domain/network node you're trying to connect could be offline at the current moment. Please try again later.	


We can see that there is no connected `domain` yet

#### 2. Launch the first `domain` to a remote machine:
1. Connect to the remote machine with `ssh -i <path_to_key> <user_name>@<address>`
2. If we don't already know the `x.x.x.x` address in step 1, we can find out the `ip` address of the vm with `ip route`. Let's say we found out that the `ip address` is `130.230.52.239`
3. Launch the `domain` to the remote machine with the command below

In [ ]:
!HAGRID_ART=false hagrid launch test_domain domain to 130.230.52.239 \
  --username=ubuntu --auth-type=key \
  --key-path=/home/dk/Desktop/projects/ssh_keys/triton_key \
  --repo=OpenMined/PySyft --branch=0.7.0 --tail


we will see something like below running:
![](./assets/5-hagrid-launch_to_ip.png)

Check the `domain` health

In [ ]:
import hagrid
hagrid.check('130.230.52.239:80')

In [ ]:
domain_client_1 = sy.login(
    url="130.230.52.239",
    port=80,
    email="info@openmined.org",
    password="changethis"
)

Alternatively, if things fail, we can just `ssh` into the `vm`, install `git`, `docker`, `hagrid`, `syft` and manually provision the `domain` with `hagrid launch to docker:80 --tail`. Then using `sy.login` etc.

#### 3. Launch the second `domain` to `GCP`

Install `gcp cli tool` and log in (note that we also need to have the billing account), create a project on gcp and run the command 

In [ ]:
!hagrid launch test-domain domain to gcp --tail

and we will be asked to provide the `project_id`, the zone of our VM, the machine type... and we also need to enable the `compute engine api`

To enable the compute engine api, go to the tab `APIs and Services`, then `Enabled APIs and Services` to enable `compute engine api`
![](./assets/5-hagrid-enable-compute-api.png)

If the code runs successfully, we will see the in the `terminal`

![](./assets/5-hagrid-gcp-running.png)

In the `gcp console`, in the tab `Compute Engine - VM instances` we can see something like below
![](./assets/5-hagrid-vm-instances.png)

Now let's check the `domain` health

In [ ]:
hagrid.check('35.192.100.214:80')

Sign in to the `domain`

In [ ]:
import syft as sy
domain_client_2 = sy.login(
    url="35.192.100.214",
    port=80,
    email="info@openmined.org",
    password="changethis"
)

Alternatively, we can create a `vm` on `gcp`, connect to the vm (https://cloud.google.com/compute/docs/connect/standard-ssh#gcloud) and then manually install `docker`, `hagrid`, `syft`, launch the domain inside the vm with `hagrid launch to docker:80 --tail` and log in with `syft.login` etc. You should be able to access the `domain` from outside with the public `ip address` of the `vm` and with the right port, like in the below picture. 

![](./assets/5-hagrid-connecting-to-vm-domain.png)

If you can't, then you need to add the `firewall rule` to allow the traffic to the `vm` from outside. Go to the tab `VPC network`, then `Firewall` and click `Create Firewall Rule` to add the rule. In the below picture, we saw the rule added
![](./assets/5-hagrid-firewall-rule.png)

#### 3. Connect each `domain` to the `network` and verify that they can see each other. They are connected via `tailscale`

Once the 2 `domains` and the `network` are up and running, we can connect the `domains` to the `network` using `tailscale`

In [ ]:
domain_client_1.apply_to_network(network)

In [ ]:
domain_client_2.apply_to_network(network)

In [19]:
network.domains

""
